In [18]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)

In [19]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [20]:
tf.keras.backend.clear_session()

In [21]:
subject = 'Santa Catarina - Consumo de Cimento (t)'
start_index = 0
split_index = 191 #Referente aos 230 anos de input 
window_size = 36
train_split = split_index + 1 - 30

In [22]:
data = pd.read_csv('2003_mo_model_input_SC.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data =data.drop([' NFSP - Fluxo Mensal (Milhões de reais)'], axis=1)
data

,Unnamed: 0,Santa Catarina - IDH,Santa Catarina - value,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Santa Catarina - Desemprego,Santa Catarina - Produção de Cimento (t),Santa Catarina - PIB - Estadual,Santa Catarina - PIB - Construção Civil,Santa Catarina - PIB - Per Capita,Santa Catarina - PIB - Preços de Mercado,Santa Catarina - Consumo de Cimento (t)
0,2003-1,0.797768,0.311113,0.724032,11.520143,1.639718,1.036534,1.772069e+08,7.330309e+06,0.969649,8.256606,20.216800,1.320065e+08,6.638363e+06,19.725168,1.221860e+08,142.456
1,2003-2,0.797853,0.312787,0.690297,11.189862,1.378899,0.993449,1.773884e+08,7.335910e+06,0.950783,8.250731,19.631031,1.321743e+08,6.642080e+06,19.732277,1.222513e+08,135.829
2,2003-3,0.797939,0.314207,0.669681,10.820792,1.924317,0.973020,1.775699e+08,7.341511e+06,0.938332,8.244856,19.400920,1.323421e+08,6.645797e+06,19.739386,1.223166e+08,133.135
3,2003-4,0.798024,0.315571,0.660494,10.417840,1.331174,0.940489,1.777514e+08,7.347112e+06,0.926401,8.238982,19.153741,1.325098e+08,6.649513e+06,19.746495,1.223818e+08,144.723
4,2003-5,0.798110,0.316880,0.648337,9.959690,1.736072,0.917493,1.779329e+08,7.352713e+06,0.951683,8.233107,18.912155,1.326776e+08,6.653230e+06,19.753604,1.224471e+08,150.523
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,2022-8,NaN,0.663849,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,155.321494,NaN,NaN,NaN,NaN,377.545
236,2022-9,NaN,0.663153,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,154.790929,NaN,NaN,NaN,NaN,370.282
237,2022-10,NaN,0.661797,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,155.428377,NaN,NaN,NaN,NaN,340.900
238,2022-11,NaN,0.660090,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,155.448086,NaN,NaN,NaN,NaN,363.900


In [23]:
input_data = data.iloc[:split_index + 1,1:-1]
mean = np.mean(input_data, axis=0)
stddev =  np.std(input_data, axis=0)
input_data = ((input_data - mean) /stddev)
input_data

,Santa Catarina - IDH,Santa Catarina - value,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Santa Catarina - Desemprego,Santa Catarina - Produção de Cimento (t),Santa Catarina - PIB - Estadual,Santa Catarina - PIB - Construção Civil,Santa Catarina - PIB - Per Capita,Santa Catarina - PIB - Preços de Mercado
0,-2.119998,-1.030889,2.723741,4.398348,2.052494,3.890153,-2.042341,-2.389042,3.122582,1.085359,-1.154511,-1.682734,-2.417084,-3.022828,-2.152611
1,-2.088248,-1.010380,2.350880,4.222509,1.242280,3.551840,-2.014760,-2.352139,2.970356,1.080851,-1.166496,-1.664640,-2.359361,-2.949421,-2.116149
2,-2.056499,-0.992988,2.123016,4.026019,2.936580,3.391423,-1.987179,-2.315236,2.869895,1.076343,-1.171204,-1.646546,-2.301638,-2.876014,-2.079688
3,-2.024750,-0.976287,2.021477,3.811492,1.094024,3.135979,-1.959598,-2.278333,2.773628,1.071835,-1.176262,-1.628452,-2.243915,-2.802607,-2.043226
4,-1.993000,-0.960251,1.887113,3.567576,2.351810,2.955412,-1.932017,-2.241431,2.977624,1.067328,-1.181205,-1.610358,-2.186191,-2.729200,-2.006765
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,0.726097,1.713104,-2.010387,-0.572934,-1.281958,0.589021,1.365911,0.389193,-1.749976,-1.126590,0.933479,1.196588,-0.578473,0.290543,1.029069
188,0.689854,1.722266,-1.870713,-0.588777,-1.358588,1.043728,1.376610,0.370392,-1.593005,-1.131368,0.953950,1.189460,-0.568205,0.281308,1.023160
189,0.653611,1.738852,-1.806230,-0.612606,-1.511565,1.387010,1.387308,0.351592,-1.351489,-1.136147,0.982622,1.182332,-0.557938,0.272072,1.017252
190,0.617368,1.755526,-1.727496,-0.640956,-1.421708,1.815728,1.398006,0.332791,-1.198492,-1.140925,1.015842,1.175204,-0.547670,0.262837,1.011344


In [24]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-12)
target_data

0      135.344
1      127.693
2      151.570
3      137.044
4      134.493
        ...   
235        NaN
236        NaN
237        NaN
238        NaN
239        NaN
Name: Santa Catarina - Consumo de Cimento (t), Length: 240, dtype: float64

In [25]:
# input para treinamento
train_input = input_data.iloc[start_index:train_split]
train_input

,Santa Catarina - IDH,Santa Catarina - value,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Santa Catarina - Desemprego,Santa Catarina - Produção de Cimento (t),Santa Catarina - PIB - Estadual,Santa Catarina - PIB - Construção Civil,Santa Catarina - PIB - Per Capita,Santa Catarina - PIB - Preços de Mercado
0,-2.119998,-1.030889,2.723741,4.398348,2.052494,3.890153,-2.042341,-2.389042,3.122582,1.085359,-1.154511,-1.682734,-2.417084,-3.022828,-2.152611
1,-2.088248,-1.010380,2.350880,4.222509,1.242280,3.551840,-2.014760,-2.352139,2.970356,1.080851,-1.166496,-1.664640,-2.359361,-2.949421,-2.116149
2,-2.056499,-0.992988,2.123016,4.026019,2.936580,3.391423,-1.987179,-2.315236,2.869895,1.076343,-1.171204,-1.646546,-2.301638,-2.876014,-2.079688
3,-2.024750,-0.976287,2.021477,3.811492,1.094024,3.135979,-1.959598,-2.278333,2.773628,1.071835,-1.176262,-1.628452,-2.243915,-2.802607,-2.043226
4,-1.993000,-0.960251,1.887113,3.567576,2.351810,2.955412,-1.932017,-2.241431,2.977624,1.067328,-1.181205,-1.610358,-2.186191,-2.729200,-2.006765
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,1.514485,1.376970,-0.214006,-0.607704,0.059280,-1.233012,1.031384,0.819304,-0.883659,-0.817925,0.890411,1.181211,-0.551568,0.108581,0.958937
158,1.496494,1.401965,-0.434717,-0.620523,0.222389,-1.299304,1.042716,0.808136,-0.950771,-0.835333,0.883759,1.187451,-0.577579,0.126542,0.968867
159,1.478503,1.424150,-0.524091,-0.631530,0.191929,-1.248662,1.054049,0.796969,-1.028465,-0.852741,0.877272,1.193692,-0.603589,0.144504,0.978796
160,1.460512,1.443460,-0.614500,-0.640320,0.385687,-1.068274,1.065381,0.785801,-1.103668,-0.870149,0.861580,1.199932,-0.629600,0.162465,0.988726


In [26]:
# Alvo para treinamento
train_target = target_data.iloc[start_index:train_split]
train_target

0      135.344
1      127.693
2      151.570
3      137.044
4      134.493
        ...   
157    211.861
158    258.760
159    211.393
160    249.531
161    231.359
Name: Santa Catarina - Consumo de Cimento (t), Length: 162, dtype: float64

In [27]:
def create_batches(t_input, t_target, window_size, start_from):
    
    X_batches = []
    y_batches = []

    train_input_values = t_input.values 

    for i in range(len(t_input) - window_size):
        
        X_window = train_input_values[i:i+window_size, :]
        y_target = t_target[start_from+i+window_size]

        X_batches.append(X_window)
        y_batches.append(y_target)

    return np.array(X_batches), np.array(y_batches)

In [28]:
reshaped_train, reshaped_target = create_batches(train_input, 
                                                 train_target, 
                                                 window_size, 
                                                 start_index)
reshaped_train.shape

(126, 36, 15)

In [29]:
# input para treinamento
test_input = input_data.iloc[train_split - window_size:split_index + 1]
test_input

,Santa Catarina - IDH,Santa Catarina - value,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Santa Catarina - Desemprego,Santa Catarina - Produção de Cimento (t),Santa Catarina - PIB - Estadual,Santa Catarina - PIB - Construção Civil,Santa Catarina - PIB - Per Capita,Santa Catarina - PIB - Preços de Mercado
126,0.730991,0.354460,0.888984,-0.460555,-1.131704,-0.368821,0.651397,0.944085,0.045243,-1.145081,1.261102,0.800980,0.503024,0.784774,0.894737
127,0.741997,0.396550,0.954254,-0.440372,-0.613483,-0.328087,0.664707,0.947319,0.061828,-1.124363,1.264193,0.819811,0.473034,0.769927,0.903886
128,0.753002,0.439226,1.045217,-0.419247,-0.542919,-0.176031,0.678017,0.950553,0.046225,-1.103645,1.264246,0.838643,0.443045,0.755079,0.913035
129,0.764008,0.479949,1.176395,-0.397019,-1.334517,-0.113037,0.691327,0.953786,0.032522,-1.082927,1.262191,0.857474,0.413055,0.740232,0.922184
130,0.775014,0.523578,1.303259,-0.376532,-1.511762,-0.022703,0.704637,0.957020,0.042757,-1.062209,1.252654,0.876306,0.383065,0.725384,0.931332
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,0.726097,1.713104,-2.010387,-0.572934,-1.281958,0.589021,1.365911,0.389193,-1.749976,-1.126590,0.933479,1.196588,-0.578473,0.290543,1.029069
188,0.689854,1.722266,-1.870713,-0.588777,-1.358588,1.043728,1.376610,0.370392,-1.593005,-1.131368,0.953950,1.189460,-0.568205,0.281308,1.023160
189,0.653611,1.738852,-1.806230,-0.612606,-1.511565,1.387010,1.387308,0.351592,-1.351489,-1.136147,0.982622,1.182332,-0.557938,0.272072,1.017252
190,0.617368,1.755526,-1.727496,-0.640956,-1.421708,1.815728,1.398006,0.332791,-1.198492,-1.140925,1.015842,1.175204,-0.547670,0.262837,1.011344


In [30]:
reshaped_test, reshaped_test_target = create_batches(test_input, 
                                                     target_data, 
                                                     window_size, 
                                                     train_split - window_size)
reshaped_test.shape 

(30, 36, 15)

In [31]:
def validation_splitter(arr, div_factor, add_factor=0):
    split_factor = len(arr) // div_factor
    positions_to_drop_index = []
    positions_to_drop = []
    for i in range(split_factor):
        pos = len(arr) - (i * div_factor + 1)
        positions_to_drop_index.append(pos)
        positions_to_drop.append(pos + add_factor)
    
    arr_droped = arr[positions_to_drop]
    arr_result = np.delete(arr, positions_to_drop_index, axis=0)
    
    return arr_result, arr_droped

In [32]:
# Rede Neural Recorrente com optmizador Estocástico
def lstm_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    train, train_val = validation_splitter(train_input, 7)
    target,target_val = validation_splitter(train_target, 7)
#     display(train.shape)
#     display(train_val.shape)
#     display(target.shape)
#     display(target_val.shape)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.LSTM(36, activation='tanh', 
                             return_sequences=True, 
                             input_shape=(reshaped_train.shape[1],
                                          reshaped_train.shape[2])),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.LSTM(180,activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train, 
                        target, 
                        epochs=10000,
                        validation_data=(train_val,
                                         target_val),
                        callbacks=[early_stopping], 
                        verbose=want_verbose)
    return model, history

In [33]:
def get_a_good_seed(train_input, train_target, test_input, test_target):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(50)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = lstm_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")
            if winner_seed == 0.0:
                return winner_seed

    return winner_seed

In [34]:
winner_seed = get_a_good_seed(reshaped_train, 
                              reshaped_target, 
                              reshaped_test, 
                              reshaped_test_target)
print(f"\n\nfinal_seed: {winner_seed}")

[2528015029, 2934895837, 2231724824, 1313385090, 3826857941, 1362781981, 1369008277, 300132105, 3436467960, 1711352831, 595913553, 244629283, 1123752532, 2965239142, 618567911, 2144218381, 1722454795, 1047623845, 3618925568, 2790565135, 1419594388, 1090656401, 2609280127, 2100556196, 1732340123, 1736812519, 1931036757, 2061733653, 3390341612, 3088480564, 561518749, 4190247986, 1504232709, 3374774175, 74625712, 1039229595, 4008027344, 2180938428, 2951791582, 3975735003, 1318494221, 4004861564, 3704515214, 650336855, 263559269, 3585810024, 4103110183, 1376097181, 157035339, 2940370890]


Step: 0 ___________________________________________
val_loss: 200.734130859375
winner_seed: 2528015029


Step: 1 ___________________________________________
val_loss: 361.9973449707031


Step: 2 ___________________________________________
val_loss: 413.3192443847656


Step: 3 ___________________________________________
val_loss: 424.4509582519531


Step: 4 ___________________________________________
val_

In [35]:
trained_model, history = lstm_model(reshaped_train, 
                                    reshaped_target, 
                                    want_verbose=1, 
                                    seed=winner_seed)

Epoch 1/10000
4/4 [==============================] - 2s 129ms/step - loss: 46494.0820 - val_loss: 1770.8436
Epoch 2/10000
4/4 [==============================] - 0s 12ms/step - loss: 1894.9719 - val_loss: 1353.9075
Epoch 3/10000
4/4 [==============================] - 0s 12ms/step - loss: 1616.2130 - val_loss: 1412.1903
Epoch 4/10000
4/4 [==============================] - 0s 11ms/step - loss: 1802.9521 - val_loss: 1445.1997
Epoch 5/10000
4/4 [==============================] - 0s 11ms/step - loss: 1892.1609 - val_loss: 1347.0161
Epoch 6/10000
4/4 [==============================] - 0s 11ms/step - loss: 1556.0947 - val_loss: 1735.3546
Epoch 7/10000
4/4 [==============================] - 0s 11ms/step - loss: 1859.0898 - val_loss: 1256.0594
Epoch 8/10000
4/4 [==============================] - 0s 10ms/step - loss: 1721.4800 - val_loss: 1379.9430
Epoch 9/10000
4/4 [==============================] - 0s 11ms/step - loss: 1742.6328 - val_loss: 1197.9182
Epoch 10/10000
4/4 [========================

4/4 [==============================] - 0s 11ms/step - loss: 395.9677 - val_loss: 460.1826
Epoch 79/10000
4/4 [==============================] - 0s 11ms/step - loss: 441.0172 - val_loss: 472.2116
Epoch 80/10000
4/4 [==============================] - 0s 11ms/step - loss: 378.8953 - val_loss: 476.0383
Epoch 81/10000
4/4 [==============================] - 0s 11ms/step - loss: 351.7984 - val_loss: 453.8081
Epoch 82/10000
4/4 [==============================] - 0s 13ms/step - loss: 334.4940 - val_loss: 502.7390
Epoch 83/10000
4/4 [==============================] - 0s 11ms/step - loss: 456.6745 - val_loss: 519.6097
Epoch 84/10000
4/4 [==============================] - 0s 11ms/step - loss: 367.2929 - val_loss: 744.0468
Epoch 85/10000
4/4 [==============================] - 0s 11ms/step - loss: 689.6960 - val_loss: 946.9017
Epoch 86/10000
4/4 [==============================] - 0s 10ms/step - loss: 715.2250 - val_loss: 1006.4973
Epoch 87/10000
4/4 [==============================] - 0s 11ms/step - 

4/4 [==============================] - 0s 11ms/step - loss: 316.5115 - val_loss: 714.6468
Epoch 156/10000
4/4 [==============================] - 0s 11ms/step - loss: 296.6213 - val_loss: 709.2680
Epoch 157/10000
4/4 [==============================] - 0s 11ms/step - loss: 332.1850 - val_loss: 637.6512
Epoch 158/10000
4/4 [==============================] - 0s 11ms/step - loss: 353.0115 - val_loss: 768.2343
Epoch 159/10000
4/4 [==============================] - 0s 11ms/step - loss: 340.8885 - val_loss: 478.3882
Epoch 160/10000
4/4 [==============================] - 0s 11ms/step - loss: 336.3362 - val_loss: 622.1423
Epoch 161/10000
4/4 [==============================] - 0s 11ms/step - loss: 592.6040 - val_loss: 585.9263
Epoch 162/10000
4/4 [==============================] - 0s 10ms/step - loss: 292.4137 - val_loss: 546.9019
Epoch 163/10000
4/4 [==============================] - 0s 10ms/step - loss: 251.9605 - val_loss: 527.8993
Epoch 164/10000
4/4 [==============================] - 0s 10ms

4/4 [==============================] - 0s 10ms/step - loss: 287.9421 - val_loss: 582.1705
Epoch 233/10000
4/4 [==============================] - 0s 11ms/step - loss: 297.5086 - val_loss: 522.9657
Epoch 234/10000
4/4 [==============================] - 0s 10ms/step - loss: 290.1164 - val_loss: 600.3417
Epoch 235/10000
4/4 [==============================] - 0s 11ms/step - loss: 395.4433 - val_loss: 515.0574
Epoch 236/10000
4/4 [==============================] - 0s 12ms/step - loss: 283.4521 - val_loss: 471.5627
Epoch 237/10000
4/4 [==============================] - 0s 11ms/step - loss: 325.5994 - val_loss: 481.9307
Epoch 238/10000
4/4 [==============================] - 0s 11ms/step - loss: 304.0716 - val_loss: 504.3709
Epoch 239/10000
4/4 [==============================] - 0s 11ms/step - loss: 299.8907 - val_loss: 507.5442
Epoch 240/10000
4/4 [==============================] - 0s 11ms/step - loss: 332.6815 - val_loss: 498.6627
Epoch 241/10000
4/4 [==============================] - 0s 11ms

4/4 [==============================] - 0s 12ms/step - loss: 286.6810 - val_loss: 495.0240
Epoch 310/10000
4/4 [==============================] - 0s 11ms/step - loss: 248.0410 - val_loss: 476.7180
Epoch 311/10000
4/4 [==============================] - 0s 12ms/step - loss: 250.9039 - val_loss: 482.3345
Epoch 312/10000
4/4 [==============================] - 0s 11ms/step - loss: 310.7865 - val_loss: 511.4738
Epoch 313/10000
4/4 [==============================] - 0s 12ms/step - loss: 262.6752 - val_loss: 464.9699
Epoch 314/10000
4/4 [==============================] - 0s 11ms/step - loss: 283.7129 - val_loss: 536.1359
Epoch 315/10000
4/4 [==============================] - 0s 12ms/step - loss: 245.5132 - val_loss: 514.4451
Epoch 316/10000
4/4 [==============================] - 0s 12ms/step - loss: 257.8657 - val_loss: 500.8641
Epoch 317/10000
4/4 [==============================] - 0s 12ms/step - loss: 259.8944 - val_loss: 463.5490
Epoch 318/10000
4/4 [==============================] - 0s 11ms

...

4/4 [==============================] - 0s 12ms/step - loss: 78.4007 - val_loss: 559.4412
Epoch 2078/10000
4/4 [==============================] - 0s 11ms/step - loss: 108.6983 - val_loss: 462.0071
Epoch 2079/10000
4/4 [==============================] - 0s 11ms/step - loss: 78.3948 - val_loss: 459.5688
Epoch 2080/10000
4/4 [==============================] - 0s 10ms/step - loss: 106.4269 - val_loss: 584.0082
Epoch 2081/10000
4/4 [==============================] - 0s 10ms/step - loss: 157.6420 - val_loss: 315.8318
Epoch 2082/10000
4/4 [==============================] - 0s 10ms/step - loss: 158.7384 - val_loss: 382.4563
Epoch 2083/10000
4/4 [==============================] - 0s 10ms/step - loss: 106.6825 - val_loss: 374.7592
Epoch 2084/10000
4/4 [==============================] - 0s 10ms/step - loss: 78.2180 - val_loss: 413.3323
Epoch 2085/10000
4/4 [==============================] - 0s 9ms/step - loss: 162.1112 - val_loss: 448.5755
Epoch 2086/10000
4/4 [==============================] - 0s

4/4 [==============================] - 0s 11ms/step - loss: 65.6984 - val_loss: 343.6727
Epoch 2155/10000
4/4 [==============================] - 0s 10ms/step - loss: 69.7747 - val_loss: 374.3379
Epoch 2156/10000
4/4 [==============================] - 0s 10ms/step - loss: 64.3658 - val_loss: 352.1769
Epoch 2157/10000
4/4 [==============================] - 0s 10ms/step - loss: 57.6021 - val_loss: 356.0836
Epoch 2158/10000
4/4 [==============================] - 0s 10ms/step - loss: 61.6234 - val_loss: 404.2311
Epoch 2159/10000
4/4 [==============================] - 0s 10ms/step - loss: 54.2360 - val_loss: 378.2126
Epoch 2160/10000
4/4 [==============================] - 0s 11ms/step - loss: 86.2225 - val_loss: 384.2654
Epoch 2161/10000
4/4 [==============================] - 0s 11ms/step - loss: 69.5618 - val_loss: 340.1295
Epoch 2162/10000
4/4 [==============================] - 0s 10ms/step - loss: 56.1767 - val_loss: 265.0832
Epoch 2163/10000
4/4 [==============================] - 0s 9ms/

4/4 [==============================] - 0s 10ms/step - loss: 58.3172 - val_loss: 241.0520
Epoch 2232/10000
4/4 [==============================] - 0s 11ms/step - loss: 48.4790 - val_loss: 327.7205
Epoch 2233/10000
4/4 [==============================] - 0s 10ms/step - loss: 51.9633 - val_loss: 321.0284
Epoch 2234/10000
4/4 [==============================] - 0s 10ms/step - loss: 64.1701 - val_loss: 336.0853
Epoch 2235/10000
4/4 [==============================] - 0s 11ms/step - loss: 68.4986 - val_loss: 321.3138
Epoch 2236/10000
4/4 [==============================] - 0s 10ms/step - loss: 57.7065 - val_loss: 303.4563
Epoch 2237/10000
4/4 [==============================] - 0s 9ms/step - loss: 56.0580 - val_loss: 352.2232
Epoch 2238/10000
4/4 [==============================] - 0s 10ms/step - loss: 58.9606 - val_loss: 261.2018
Epoch 2239/10000
4/4 [==============================] - 0s 10ms/step - loss: 53.0702 - val_loss: 321.1114
Epoch 2240/10000
4/4 [==============================] - 0s 11ms/

Epoch 2308/10000
4/4 [==============================] - 0s 10ms/step - loss: 101.1943 - val_loss: 377.8865
Epoch 2309/10000
4/4 [==============================] - 0s 11ms/step - loss: 106.0165 - val_loss: 382.4672
Epoch 2310/10000
4/4 [==============================] - 0s 10ms/step - loss: 173.3493 - val_loss: 387.1684
Epoch 2311/10000
4/4 [==============================] - 0s 11ms/step - loss: 99.2697 - val_loss: 430.3345
Epoch 2312/10000
4/4 [==============================] - 0s 11ms/step - loss: 84.1783 - val_loss: 400.2055
Epoch 2313/10000
4/4 [==============================] - 0s 12ms/step - loss: 106.7427 - val_loss: 382.7889
Epoch 2314/10000
4/4 [==============================] - 0s 12ms/step - loss: 121.3202 - val_loss: 382.6761
Epoch 2315/10000
4/4 [==============================] - 0s 11ms/step - loss: 89.2964 - val_loss: 397.3476
Epoch 2316/10000
4/4 [==============================] - 0s 12ms/step - loss: 92.7022 - val_loss: 365.3326
Epoch 2317/10000
4/4 [===================

4/4 [==============================] - 0s 9ms/step - loss: 87.1026 - val_loss: 415.5052
Epoch 2386/10000
4/4 [==============================] - 0s 10ms/step - loss: 67.3848 - val_loss: 466.4722
Epoch 2387/10000
4/4 [==============================] - 0s 11ms/step - loss: 163.4100 - val_loss: 331.3796
Epoch 2388/10000
4/4 [==============================] - 0s 10ms/step - loss: 63.0001 - val_loss: 378.2874
Epoch 2389/10000
4/4 [==============================] - 0s 10ms/step - loss: 78.5974 - val_loss: 370.8950
Epoch 2390/10000
4/4 [==============================] - 0s 10ms/step - loss: 78.6341 - val_loss: 369.1332
Epoch 2391/10000
4/4 [==============================] - 0s 9ms/step - loss: 68.1606 - val_loss: 341.5116
Epoch 2392/10000
4/4 [==============================] - 0s 10ms/step - loss: 75.9852 - val_loss: 311.6378
Epoch 2393/10000
4/4 [==============================] - 0s 10ms/step - loss: 109.0492 - val_loss: 326.5862
Epoch 2394/10000
4/4 [==============================] - 0s 11ms

In [36]:
def mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    results_data = []

    for i in range(len(test_target)):
        prediction = model.predict(test_input[i:i+1])
        target = test_target[start_target + i]
        error = np.abs(prediction - target)
        errors.append(error)
        error_percent.append(error/target)
        results_data.append([f"Month-{i + 1}", 
                             prediction[0][0], 
                             target, 
                             error[0][0]])

    df_results = pd.DataFrame(results_data, columns=["Month", "Prediction", "Target", "Error"])

    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return df_results, mae, mape

In [37]:
df_results, mae, mape = mae_mape_calculator(trained_model, 
                                            reshaped_test, 
                                            reshaped_test_target, 
                                            start_index)
pd.set_option('display.max_columns', None)
df_results.T

1/1 [==============================] - 0s 14ms/step


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
Month,Month-1,Month-2,Month-3,Month-4,Month-5,Month-6,Month-7,Month-8,Month-9,Month-10,Month-11,Month-12,Month-13,Month-14,Month-15,Month-16,Month-17,Month-18,Month-19,Month-20,Month-21,Month-22,Month-23,Month-24,Month-25,Month-26,Month-27,Month-28,Month-29,Month-30
Prediction,232.957932,233.380447,233.231964,233.491714,233.561569,233.549438,233.599838,233.863205,233.978394,233.998535,234.000824,234.221115,233.556229,234.369797,234.851852,234.245255,233.873398,233.609863,236.057632,238.800507,238.472122,240.787003,241.284927,240.461288,239.518234,237.229172,236.784561,235.023178,235.252594,234.719238
Target,265.999,277.169,245.824,244.227,263.751,211.136,219.002,234.275,249.92,246.326,198.778,285.97,262.597,277.811,235.885,272.671,278.592,214.658,252.7,246.146,255.873,273.299,270.918,256.605,303.526,299.284,284.55,307.576,281.317,226.233
Error,33.041061,43.788559,12.592041,10.735291,30.189438,22.413437,14.597839,0.411789,15.941605,12.327469,35.222824,51.748886,29.040756,43.441208,1.033142,38.425735,44.718613,18.951859,16.642365,7.34549,17.400879,32.512009,29.633072,16.143723,64.007767,62.054825,47.765427,72.552811,46.064392,8.486237


In [38]:
display(mae)
display(mape)

29.307682

0.10925533

In [42]:
def year_mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    
    target_sum = 0
    prediction_sum = 0
    
    for i in range(len(test_target)):
        if i % 12 == 0 and i != 0:
            error = np.abs(target_sum - prediction_sum)
            errors.append(error)
            error_percent.append(error / target_sum)
            print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
            target_sum = 0
            prediction_sum = 0
            
        prediction = model.predict(test_input[i:i+1])
        target_sum += test_target[start_target + i]
        prediction_sum += prediction
        
    error = np.abs(target_sum - prediction_sum)
    errors.append(error)
    error_percent.append(error / target_sum)
    print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
        
    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return errors, mae, mape

In [43]:
errors, mae, mape = year_mae_mape_calculator(trained_model, 
                                             reshaped_test, 
                                             reshaped_test_target,
                                             start_index)
display(errors)
display(mae)
display(mape)

1/1 [==============================] - 0s 15ms/step
Ano-0: |Prediction[[2803.8347]] - Target[2942.3769999999995]| =  Error: [[138.54224]]; MAPE:[[0.04708514]]
1/1 [==============================] - 0s 18ms/step
Ano-0: |Prediction[[2840.3699]] - Target[3097.755]| =  Error: [[257.385]]; MAPE:[[0.0830876]]
1/1 [==============================] - 0s 14ms/step
Ano-5: |Prediction[[1418.527]] - Target[1702.4859999999999]| =  Error: [[283.95898]]; MAPE:[[0.1667908]]


[array([[138.54224]], dtype=float32),
 array([[257.385]], dtype=float32),
 array([[283.95898]], dtype=float32)]

226.62874

0.09898785